In [5]:
import spacy
from spacy import displacy
from spacy.pipeline import EntityRuler
from spacy.pipeline import merge_entities

In [6]:
nlp = spacy.load("en_core_web_sm")
# patterns = [
#     {"label":"DATE", "pattern":[{"TEXT": {"REGEX": "^([0-2][0-9]|(3)[0-1])(\/)(((0)[0-9])|((1)[0-2]))(\/)\d{4}$"}}]}
#     {"label":"COL-DET", "pattern":}
# ]
# ruler.add_patterns(patterns)
# ruler.to_disk("../data/patterns.jsonl")

ruler = EntityRuler(nlp, overwrite_ents=True).from_disk("../data/patterns.jsonl")

In [7]:
nlp.add_pipe(ruler)
nlp.add_pipe(merge_entities)

In [12]:
inputQs = []
with open('../data/queslist.txt', 'r') as f:
    x = f.readlines()
    for t in range(0,14000,50):
        tx = x[t].strip()
        inputQs.append(tx)
    

In [30]:
displacy.render(doc, options={'fine-grained': True})

In [63]:
def Hub(questions):
    for NLquery in questions:
        doc = nlp(NLquery)
#         print("Q: ", NLquery)
        displacy.render(doc, style='ent')
        print("Tokens:")
        for token in doc:
            print("{:20} {:20} {:20} {:20}".format(token.text, token.pos_, token.tag_, token.dep_))
        print()
        print("Noun Chunks:")
        for chunk in doc.noun_chunks:
            print("{:25} {:25} {:25} {:25}".format(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text))
        # Finding a verb with a subject from below — good
        verbs = set()
        nchunk = set()
        print()
        print("Entities:")
        for ent in doc.ents:
            print("{:25} {:25}".format(ent.text, ent.label_))
        print()
        for possible_subject in doc.noun_chunks:
            if (possible_subject.root.dep == nsubj or possible_subject.root.dep == nsubjpass) and possible_subject.root.head.pos == VERB:
                verbs.add(possible_subject.root.head)
#                 nchunk.add(possible_subject.)
        print(verbs)

In [64]:
inputQs = ['Who was the winner between Mum Indians and Super Kings at Mumbai?',
 'Who was the loser between Mum Indians and Super Kings at Mumbai?',
 'Who was the winner between Mum Indians and Super Kings on 07/04/2018?',
 'Who was the loser between Mum Indians and Super Kings on 07/04/2018?',
 'Where was the match between Mum Indians & Super Kings held at Mumbai?',
 'When was the match between Mum Indians & Super Kings held on 07/04/2018?',
 'How many runs were scored by RG Sharma in the match between Mum Indians and Super Kings at Mumbai?',
 'How many runs were scored by RG Sharma in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many fours were scored by RG Sharma in the match between Mum Indians and Super Kings at Mumbai?',
 'How many fours were scored by RG Sharma in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many sixes were scored by RG Sharma in the match between Mum Indians and Super Kings at Mumbai?',
 'How many sixes were scored by RG Sharma in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many runs were scored against DL Chahar in the match between Mum Indians and Super Kings at Mumbai?',
 'How many runs were scored against DL Chahar in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many fours were scored against DL Chahar in the match between Mum Indians and Super Kings at Mumbai?',
 'How many fours were scored against DL Chahar in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many sixes were scored against DL Chahar in the match between Mum Indians and Super Kings at Mumbai?',
 'How many sixes were scored against DL Chahar in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many overs did DL Chahar bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many overs did DL Chahar bowl in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many no balls did DL Chahar bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many no balls did DL Chahar bowl in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many wide balls did DL Chahar bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many wide balls did DL Chahar bowl in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many runs were scored against SR Watson in the match between Mum Indians and Super Kings at Mumbai?',
 'How many runs were scored against SR Watson in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many fours were scored against SR Watson in the match between Mum Indians and Super Kings at Mumbai?',
 'How many fours were scored against SR Watson in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many sixes were scored against SR Watson in the match between Mum Indians and Super Kings at Mumbai?',
 'How many sixes were scored against SR Watson in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many overs did SR Watson bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many overs did SR Watson bowl in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many no balls did SR Watson bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many no balls did SR Watson bowl in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many wide balls did SR Watson bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many wide balls did SR Watson bowl in the match between Mum Indians and Super Kings on 07/04/2018?']

In [65]:
Hub(inputQs)

Tokens:
Who                  PRON                 WP                   attr                
was                  VERB                 VBD                  ROOT                
the                  DET                  DT                   det                 
winner               NOUN                 NN                   attr                
between              ADP                  IN                   prep                
Mum Indians          PROPN                NNPS                 pobj                
and                  CCONJ                CC                   cc                  
Super Kings          NOUN                 NNS                  conj                
at                   ADP                  IN                   prep                
Mumbai               PROPN                NNP                  pobj                
?                    PUNCT                .                    punct               

Noun Chunks:
Who                       Who                       at

Tokens:
Who                  PRON                 WP                   attr                
was                  VERB                 VBD                  ROOT                
the                  DET                  DT                   det                 
loser                NOUN                 NN                   attr                
between              ADP                  IN                   prep                
Mum Indians          PROPN                NNPS                 pobj                
and                  CCONJ                CC                   cc                  
Super Kings          NOUN                 NNS                  conj                
at                   ADP                  IN                   prep                
Mumbai               PROPN                NNP                  pobj                
?                    PUNCT                .                    punct               

Noun Chunks:
Who                       Who                       at

Tokens:
Who                  PRON                 WP                   attr                
was                  VERB                 VBD                  ROOT                
the                  DET                  DT                   det                 
winner               NOUN                 NN                   attr                
between              ADP                  IN                   prep                
Mum Indians          PROPN                NNPS                 pobj                
and                  CCONJ                CC                   cc                  
Super Kings          NOUN                 NNS                  conj                
on                   ADP                  IN                   prep                
07/04/2018           NUM                  CD                   punct               
?                    PUNCT                .                    punct               

Noun Chunks:
Who                       Who                       at

Tokens:
Who                  PRON                 WP                   attr                
was                  VERB                 VBD                  ROOT                
the                  DET                  DT                   det                 
loser                NOUN                 NN                   attr                
between              ADP                  IN                   prep                
Mum Indians          PROPN                NNPS                 pobj                
and                  CCONJ                CC                   cc                  
Super Kings          NOUN                 NNS                  conj                
on                   ADP                  IN                   prep                
07/04/2018           NUM                  CD                   punct               
?                    PUNCT                .                    punct               

Noun Chunks:
Who                       Who                       at

Tokens:
Where                ADV                  WRB                  advmod              
was                  VERB                 VBD                  ROOT                
the                  DET                  DT                   det                 
match                NOUN                 NN                   nsubj               
between              ADP                  IN                   prep                
Mum Indians          PROPN                NNP                  pobj                
&                    CCONJ                CC                   cc                  
Super Kings          NOUN                 NNS                  conj                
held                 VERB                 VBN                  acl                 
at                   ADP                  IN                   prep                
Mumbai               PROPN                NNP                  pobj                
?                    PUNCT                .                    punct

...

Tokens:
How                  ADV                  WRB                  advmod              
many                 ADJ                  JJ                   amod                
overs                NOUN                 NNS                  nsubj               
did                  VERB                 VBD                  aux                 
SR Watson            PROPN                NNP                  compound            
bowl                 NOUN                 NN                   ROOT                
in                   ADP                  IN                   prep                
the                  DET                  DT                   det                 
match                NOUN                 NN                   pobj                
between              ADP                  IN                   prep                
Mum Indians          PROPN                NNPS                 pobj                
and                  CCONJ                CC                   cc   

Tokens:
How                  ADV                  WRB                  advmod              
many                 ADJ                  JJ                   amod                
no                   DET                  DT                   det                 
balls                NOUN                 NNS                  nsubj               
did                  VERB                 VBD                  ROOT                
SR Watson            PROPN                NNP                  compound            
bowl                 NOUN                 NN                   dobj                
in                   ADP                  IN                   prep                
the                  DET                  DT                   det                 
match                NOUN                 NN                   pobj                
between              ADP                  IN                   prep                
Mum Indians          PROPN                NNPS                 pobj 

Tokens:
How                  ADV                  WRB                  advmod              
many                 ADJ                  JJ                   amod                
no                   DET                  DT                   det                 
balls                NOUN                 NNS                  nsubj               
did                  VERB                 VBD                  ROOT                
SR Watson            PROPN                NNP                  compound            
bowl                 NOUN                 NN                   dobj                
in                   ADP                  IN                   prep                
the                  DET                  DT                   det                 
match                NOUN                 NN                   pobj                
between              ADP                  IN                   prep                
Mum Indians          PROPN                NNPS                 pobj 

Tokens:
How                  ADV                  WRB                  advmod              
many                 ADJ                  JJ                   amod                
wide                 ADJ                  JJ                   amod                
balls                NOUN                 NNS                  nsubj               
did                  VERB                 VBD                  ROOT                
SR Watson            PROPN                NNP                  compound            
bowl                 NOUN                 NN                   dobj                
in                   ADP                  IN                   prep                
the                  DET                  DT                   det                 
match                NOUN                 NN                   pobj                
between              ADP                  IN                   prep                
Mum Indians          PROPN                NNPS                 pobj 

Tokens:
How                  ADV                  WRB                  advmod              
many                 ADJ                  JJ                   amod                
wide                 ADJ                  JJ                   amod                
balls                NOUN                 NNS                  nsubj               
did                  VERB                 VBD                  ROOT                
SR Watson            PROPN                NNP                  compound            
bowl                 NOUN                 NN                   dobj                
in                   ADP                  IN                   prep                
the                  DET                  DT                   det                 
match                NOUN                 NN                   pobj                
between              ADP                  IN                   prep                
Mum Indians          PROPN                NNPS                 pobj 